In [ ]:
%pip install --user numpy
%pip install --user matplotlib

In [3]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [262]:
class MultiLayerPerceptronClassifier:
    
    def __init__(self, hidden_layer, suppress_output = False):
        self.W = None
        self.b = None
        self.suppress_output = False
        self.hidden_layer = hidden_layer
        self.outputs = None
        self.A = None
        
    def fit(self, X, y):
        #Se hará la suposición que todas las filas de X tienen el mismo número de columnas.
        self.build_network(len(X[0]))
        
        print("FF: %s " % self.feed_forward(X[math.floor(random.random()*len(X))]))

    
    #input_length esero entero positivo mayor a 1 que representa la cantidad de atributos de cada
    #dato de entrada (sin incluir el que se busca predecir).
    #hidden_dimensions es una lista que contiene las dimensiones de las capas escondidas de la red
    #si hidden_dimensions es [3,3,3] se crearán tres capas escondidas de 
    def build_network(self, input_length):
        
        dimensions = self.hidden_layer                   
        
        #W matriz de pesos. 3D, primera dimensión capa, Segunda dimensión neurona, tercera dimensión pesos. 
        #Entonces, W[0][0] contendría el arreglo con los pesos de la neurona 0 de la capa 0.
        W = []  
        for i in range(len(dimensions)): 
            W.append([])
        
        
        #b es un arreglo 2D que contiene el bias de cada neurona b[0][0] contiene el bias de la neurona 0 
        #de la capa 0
        b = []
        for i in range(len(dimensions)): 
            b.append([])
        
        
        #todas las neuronas en la primera capa deben tener input_lenght pesos. De ahí en adelante, 
        #cada neurona en la capa j tendrá len(W[j-1]) pesos.
        layer_width = dimensions[0]
        
        for i in range(layer_width):
            W[0].append([])
            W[0][i] = [random.uniform(-1,1) for j in range(input_length)]
            b[0].append(0)
            b[0][i] = random.uniform(-1,1)
            W[0][i] = np.array([W[0][i]])
            

            
        #Inicialización aleatoria de pesos y bias entre -1 y 1
        for i in range(1, len(dimensions)):
            layer_width = dimensions[i]
            for j in range(layer_width):
                #se inicializan n valores aleatorios entre -1 y 1 donde n = len(W[i-1])
                W[i].append([])
                W[i][j] = [random.uniform(-1,1) for j in range(len(W[i-1]))]
                b[i].append(0)
                #se crea un bias aleatorio para la neurona
                b[i][j] = random.uniform(-1,1) 
            
                W[i][j] = np.array([W[i][j]])
                
        
        #Ahora se insertan los pesos y bias al output unit
        W.append([])
        W[len(W) - 1].append([])
        b.append([])
        b[len(W) - 1].append(0)
        W[len(W) - 1][0] = [random.uniform(-1,1) for j in range(len(W[len(W)-2]))]
        W[len(W) - 1][0] = np.array([W[len(W) - 1][0]])
        b[len(W) - 1][0] = random.uniform(-1,1)
            
        self.W = W
        self.b = b
        
        if not self.suppress_output:
            print("Network Built")
            print("W")
            print(W)
            print("b")
            print(b)
            print("--------------------------------------------------------------------")
            
    #X es una entrada
    #W matriz de pesos. 3D, primera dimensión capa, Segunda dimensión neurona, tercera dimensión pesos. 
    #Entonces, W[0][0] contendría el arreglo con los pesos de la neurona 0 de la capa 0.
    #l será el número de capas.
    #b es un arreglo 2D que contiene el bias de cada neurona b[0][0] contiene el bias de la neurona 0 
    #de la capa 0.
    #Algoritmo se basa en el algoritmo 6.3 de GoodFellow et al 
    #y en el algoritmo de la página 217 de Grus en Data Science from Scratch
    def feed_forward(self, x):
        
        l = len(self.hidden_layer)
        
        #Salidas de cada neurona. Primera dimensión capa, segunda neuronas
        #outputs[0][0] representa la entrada 0 de X. 
        #outputs[1][0] representa la salida 0 de la primera capa escondida.
        
        
        
        outputs = [np.array([x])]
        A = []
        for layer in range(1,l):
            
            layer_out = []
            A_out = []
            for neuron in range(len(self.W[layer])):
                #a es el resultado de las salidas de la capa anterior por los pesos de la capa actual.
                
                X = outputs[layer-1].transpose()
                W = self.W[layer-1][neuron]
                
                a = np.dot(W,X)[0][0] + self.b[layer][neuron]
                layer_out.append(self.relu(a))
                A_out.append(a)
                
            outputs.append(np.array([layer_out]))
            A.append(np.array([A_out]))
            
        #Ahora se computa el output del output layer. 
        X = outputs[len(self.W)-2].transpose()
        W = self.W[len(self.W)-1][0]
        print("W AND X")
        print(W)
        print(X)
        np.dot(W,X)[0][0]
        self.b[len(self.W)-1][0]
        a = np.dot(W,X)[0][0] + self.b[len(self.W)-1][0]
        outputs.append(self.sigmoid(a))
        A.append(a)
        
        if not self.suppress_output:
            print("----------------------Feed_forward outputs--------------------------")
            print(outputs)
            print("----------------------Feed_forward A's------------------------------")
            print(A)
            print("--------------------------------------------------------------------")
        
        
        #Retorna el sigmoide del último output. Esto representa la probabilidad de que 
        #x pertenezca a la clase 1.
        self.outputs = outputs
        self.A = A
        return outputs[len(outputs)-1]
    
    #Relu para hidden units
    def relu(self, z):
        return max(0,z)
    
    #Sigmoide para Output unit
    def sigmoid(self, x):
        if x < 0:
            return 1 - 1/(1+math.exp(x))
        else:
            return 1/(1+math.exp(-x))
    
    def relu_derivative(self, z): 
        if z > 0:
            return 1
        else:
            return 0
    
    def back_propagation(self, e):
        #Activación traspuesta producto raro W transpuesta producto error del siguiente. 
        #Matriz bidimensional que contiene los errores de las capas escondidas. 
        #Primera dimensión capa, segunda neurona.
        
        delta = []
        
        for i in range(len(self.hidden_layer)):
            
        
        
        return None

In [196]:
data_matrix = np.loadtxt(open("./msd_genre_dataset/fixed_ds.csv", "r"), delimiter=",", skiprows=0)



print("Filas de la matriz: " + str(len(data_matrix)))
print("Columnas de la matriz: " + str(len(data_matrix[0])))

y = data_matrix[:,len(data_matrix[0])-1]
X = np.delete(data_matrix, len(data_matrix[0])-1,1)

y = y.astype(int)


print(X)
print(y)


#Se intenta estandarizar X para lograr mejor desempeño. Sin embargo, no parece funcionar.
X = MinMaxScaler().fit_transform(X)

#Los datos del set de datos están agrupados por género. Es decir, primero están todas las filas que corresponden
#a 1 y después todas las que corresponden a -1. Se hace un shuffle para que, más tarde, en cross-validation
#no se creen unos modelos que predigan únicamente una clase.
np.random.shuffle(data_matrix)


print("X shape" + str(X.shape))
print("y shape" + str(y.shape))

Filas de la matriz: 8330
Columnas de la matriz: 31
[[-18.996       89.147        1.         ... 233.20616681 261.85070337
  240.83417734]
 [-19.347      125.825        4.         ... 227.2751235  261.64357048
  332.35653566]
 [ -9.472      121.707        4.         ... 549.49321044 481.14904868
  442.66313626]
 ...
 [ -9.494       88.976        4.         ... 519.52773394 538.89585608
  313.77593362]
 [ -7.617       67.929        3.         ... 591.85304812 598.05409088
  443.89380682]
 [-11.774       85.176        3.         ... 659.32142175 531.85019809
  607.21596134]]
[ 1  1  1 ... -1 -1 -1]
X shape(8330, 30)
y shape(8330,)


In [263]:
mlp = MultiLayerPerceptronClassifier(hidden_layer = [3, 3, 3], suppress_output = False)

mlp.fit(X, y)

Network Built
W
[[array([[ 0.20404593, -0.9418346 , -0.78360462, -0.10668519,  0.4818155 ,
         0.66538047,  0.71790641,  0.30737451, -0.17846904, -0.84924797,
        -0.50004533, -0.4560268 ,  0.14124846,  0.65037077, -0.92330598,
        -0.08337044, -0.00682228, -0.50977811, -0.41016895,  0.05702923,
        -0.09608899,  0.24522899, -0.21826192,  0.62454997, -0.10225758,
        -0.59927718,  0.36533433,  0.78862051,  0.2545183 ,  0.80249054]]), array([[-0.84850357,  0.04744941,  0.21744366,  0.66675071,  0.44971991,
        -0.93947107, -0.1160988 , -0.04195025, -0.47762897,  0.04926024,
        -0.45527995, -0.78896355, -0.49554908,  0.76925949,  0.34440754,
        -0.0533385 ,  0.24558756, -0.62867312,  0.163873  , -0.03228492,
        -0.43995764, -0.99369471, -0.64750272, -0.4290368 ,  0.22823547,
        -0.63329734,  0.06641409,  0.62022267,  0.91807511,  0.8571302 ]]), array([[-0.55319969, -0.63143133, -0.57615056, -0.16564682,  0.26162562,
         0.01608506,  0.113